In [ ]:
import sqlite3
import pandas as pd

query = """
WITH SocialContext AS (
    SELECT 
        frame_number,
        video_id,
        CASE
            -- Check if both adult (1,3) AND child (0,2) are present
            WHEN MAX(CASE WHEN object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            AND MAX(CASE WHEN object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'with both children and adults'
            
            -- Check if only adult (face or person) is present
            WHEN MAX(CASE WHEN object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            THEN 'with an adult'
            
            -- Check if only child (face or person) is present
            WHEN MAX(CASE WHEN object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'with another child'
            
            -- If none of above, person is alone
            ELSE 'alone'
        END as social_context
    FROM Detections
    GROUP BY frame_number, video_id
),
ObjectPresence AS (
    SELECT 
        frame_number,
        video_id,
        MAX(CASE WHEN object_class IN (5,6,7,8,9,10) THEN 1 ELSE 0 END) as has_object,
        STRING_AGG(
            CASE 
                WHEN object_class = 5 THEN 'book'
                WHEN object_class = 6 THEN 'toy'
                WHEN object_class = 7 THEN 'kitchenware'
                WHEN object_class = 8 THEN 'screen'
                WHEN object_class = 9 THEN 'food'
                WHEN object_class = 10 THEN 'other_object'
            END,
            ', ' 
        ) as object_types
    FROM Detections
    WHERE object_class IN (5,6,7,8,9,10)
    GROUP BY frame_number, video_id
)

SELECT DISTINCT
    f.frame_id,
    f.video_id,
    COALESCE(op.object_types, 'none') as object_type,
    v.age_at_recording,
    COALESCE(sc.social_context, 'alone') as social_context,
    COALESCE(op.has_object, 0) as has_object
FROM 
    Frames f
    LEFT JOIN Videos v ON f.video_id = v.video_id
    LEFT JOIN SocialContext sc ON f.frame_number = sc.frame_number AND f.video_id = sc.video_id
    LEFT JOIN ObjectPresence op ON f.frame_number = op.frame_number AND f.video_id = op.video_id
ORDER BY 
    f.video_id, f.frame_id;
"""
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    df = pd.read_sql_query(query, conn)

print(f"Retrieved {len(df)} rows")
display(df.head())

DatabaseError: Execution failed on sql '
WITH SocialContext AS (
    SELECT 
        frame_number,
        video_id,
        CASE
            -- Check if both adult (1,3) AND child (0,2) are present
            WHEN MAX(CASE WHEN object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            AND MAX(CASE WHEN object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'with both children and adults'
            
            -- Check if only adult (face or person) is present
            WHEN MAX(CASE WHEN object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            THEN 'with an adult'
            
            -- Check if only child (face or person) is present
            WHEN MAX(CASE WHEN object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'with another child'
            
            -- If none of above, person is alone
            ELSE 'alone'
        END as social_context
    FROM Detections
    GROUP BY frame_number, video_id
),
ObjectPresence AS (
    SELECT 
        frame_